In [1]:
import sys
import pandas as pd
from pandas import DataFrame
# import pyodbc 
import pymysql
import os
import re
import numpy as np
from fractions import Fraction
import json
import chardet
from ckiptagger import WS, POS
from ckiptagger import construct_dictionary
from numba import jit

#---------Vote---------#
import distance                                             #Levenshtein distance演算
import jieba                                                #結巴
from rank_bm25 import BM25Okapi                             #bm25演算
from sklearn.feature_extraction.text import CountVectorizer #TFIDF演算
import synonyms                                        #synonyms演算
from gensim.models import *                            #文字轉向量
from sklearn.metrics.pairwise import cosine_similarity #餘弦


[jieba] default dict file path ..\data\vocab.txt


Building prefix dict from C:\Users\jenny\anaconda3\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\jenny\AppData\Local\Temp\jieba.u293ec38435f0667c956027d0ce0cade2.cache


>> Synonyms load wordseg dict [C:\Users\jenny\anaconda3\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.584 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [C:\Users\jenny\anaconda3\lib\site-packages\synonyms\data\stopwords.txt] ...
[Synonyms] on loading vectors [C:\Users\jenny\anaconda3\lib\site-packages\synonyms\data\words.vector.gz] ...


In [2]:
# 資料庫連結
db_settings = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "1qaz2wsx3EDC",
    "db": "Nutrition"
}
conn = pymysql.connect(**db_settings)
# 建立Cursor物件
cursor=conn.cursor()

dirname = 'nlp_txt/'

ws = WS("C:/5Python/API/data")
pos = POS("C:/5Python/API/data")

In [3]:
#量詞字典
def getWS_Dict():
    global dict_temp
    dict_temp = {}

    file = open('WS_Dict.txt','r',encoding='utf-8')

    for line in file.readlines():
        line = line.strip()
        k = line.split(' ')[0]
        v = line.split(' ')[1]
        dict_temp[k] = v

    file.close()

    global dict_for_CKIP
    dict_for_CKIP = construct_dictionary(dict_temp)

    return
getWS_Dict()

# 數字處理
chinese_number_dict = {'半':0.5, '一':1, '七':7, '萬':10000, '三':3, '九':9,'兩':2, '二':2, '五':5, '八':8, '六':6, '十':10,'三':3, '千':1000, '四':4, '百':100, '零':0,"億":100000000}
not_in_decimal = "十百千萬億點"
all_Num = not_in_decimal
for d in chinese_number_dict:
    all_Num += d
def ch2num(chstr):
    if '點' not in chstr:
        return ch2round(chstr)
    splits = chstr.split("點")
    if len(splits) != 2:
        return splits
    rount = ch2round(splits[0])
    decimal = ch2decimal(splits[-1])
    if rount is not None and decimal is not None:
        return float(str(rount) + "." + str(decimal))
    else:
        return None

def ch2round(chstr):
    no_op = True
    if len(chstr) >= 2:
        for i in chstr:
            if i in not_in_decimal:
                no_op = False
    else:
        no_op = False
    if no_op:
        return ch2decimal(chstr)

    result = 0
    now_base = 1
    big_base = 1
    big_big_base = 1
    base_set = set()
    chstr = chstr[::-1]
    for i in chstr:
        if i not in chinese_number_dict:
            return None
        if chinese_number_dict[i] >= 10:
            if chinese_number_dict[i] > now_base:
                now_base = chinese_number_dict[i]
            elif now_base >= chinese_number_dict["萬"] and now_base < chinese_number_dict["億"] and chinese_number_dict[i] > big_base:
                now_base = chinese_number_dict[i] * chinese_number_dict["萬"]
                big_base = chinese_number_dict[i]
            elif now_base >= chinese_number_dict["億"] and chinese_number_dict[i] > big_big_base:
                now_base = chinese_number_dict[i] * chinese_number_dict["億"]
                big_big_base = chinese_number_dict[i]
            else:
                return None
        else:
            if now_base in base_set and chinese_number_dict[i] != 0:
                return None
            result = result + now_base * chinese_number_dict[i]
            base_set.add(now_base)
    if now_base not in base_set:
        result = result + now_base * 1
    return result

def ch2decimal(chstr):
    result = ""
    for i in chstr:
        if i in not_in_decimal:
            return None
        if i not in chinese_number_dict:
            return None
        result = result + str(chinese_number_dict[i])
    return int(result)
def print_word_pos_sentence(word_sentence, pos_sentence):
    dict_list = list(dict_temp.keys())
    jsonstr = []
    assert len(word_sentence) == len(pos_sentence)
    flag = 0
    for word, pos in zip(word_sentence, pos_sentence):
        for d in dict_list:
            if word == d and word not in all_Num:
                pos = "Nf"
            s1 = WsTagger(word, pos)
        jsonstr.append(json.dumps(s1.__dict__,ensure_ascii=False))
        flag += 1
    return jsonstr

In [21]:
sentence = "我今天吃了一盤炒空心菜"

In [22]:
class WsTagger:
    def __init__(self, label, Pos):
        self.label = label
        self.Pos = Pos
class return_List:
    def __init__(self, lists, Quantity):
        self.lists = lists
        self.Quantity = Quantity
class getWS:
    def __init__(self, Name, Unit, Weight):
        self.Name = Name
        self.Unit = Unit      
        self.Weight = Weight 
def WSandPOS(sentence):

    sentence_list = []
    sentence_list =re.split(r'，|,|\+|、| |加|和|配|跟|及',sentence.replace('。',''))
    while '' in sentence_list:
        sentence_list.remove('')
    lists = []
    Json_Result_List = []
    Quantity = []
    word_sentence_list = ws(
        sentence_list,
        sentence_segmentation = True, # To consider delimiters
        recommend_dictionary = dict_for_CKIP, # words in this dictionary are encouraged
    )
    pos_sentence_list = pos(word_sentence_list)
    
    for i, sentence in enumerate(sentence_list):
        Num = "1"
        Quan = "份"
        lists = print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        flag = 0
        getL = []
        for l in lists:
            getL.append(json.loads(l))
        for item in getL:
            if item['Pos'] == "Nf":
                Quan = item['label']
            elif item['Pos'] == "Neu" or item['Pos'] == "Neqa":
                Num = item['label']


        if not Num.isdigit() and "/" not in Num and "." not in Num and "分之" not in Num:
            try:
                Num = ch2num(Num)
            except:
                Num = "1"
        elif "/" in Num:
            Num = Fraction(Num)
            Num = round(float(Num),2)
        elif "分之" in Num:
            List_Num = Num.split("分之")
            Num_0 = ch2num(List_Num[0])
            Num_1 = ch2num(List_Num[1])
            Num = str(Num_1)+"/"+str(Num_0)
            Num = Fraction(Num)
            Num = round(float(Num),2)
        JsonS = getWS(getL, Quan, Num)
        getJsonS = json.dumps(JsonS.__dict__,ensure_ascii=False)
        Json_Result_List.append(json.loads(getJsonS))
    # JsonResult = json.dumps(Json_Result_List,ensure_ascii=False)
    return Json_Result_List
WSandPOS_result = WSandPOS(sentence)
WSandPOS(sentence)

[{'Name': [{'label': '我', 'Pos': 'Nh'},
   {'label': '今天', 'Pos': 'Nd'},
   {'label': '吃', 'Pos': 'VC'},
   {'label': '了', 'Pos': 'Di'},
   {'label': '一', 'Pos': 'Neu'},
   {'label': '盤', 'Pos': 'Nf'},
   {'label': '炒', 'Pos': 'VC'},
   {'label': '空心菜', 'Pos': 'Na'}],
  'Unit': '盤',
  'Weight': 1}]

In [23]:
def CALC_Weight(WSandPOS_result):
    for i in WSandPOS_result:
        print()
        cursor.execute('SELECT Value FROM quantifier WHERE Name = \'' + str(i['Unit']) + '\'')
        get_value = cursor.fetchone()
#         print(type(get_value))
#         print(type(i['Weight']))
        Total_Weight = get_value[0] * i['Weight']
    return Total_Weight
Total_Weight = CALC_Weight(WSandPOS_result)
CALC_Weight(WSandPOS_result)

500.0

In [24]:
def getData():
    global corpus_All
    global corpus_Ingre
    corpus_All = []
    corpus_Ingre = []

    cursor.execute('SELECT Name FROM Recipe')
    for row in cursor:
        corpus_All.append(row[0])
    conn.commit()

    cursor.execute('SELECT Name, TrivialName FROM Ingre')
    for row in cursor:
        if row[1] is not None:
            TrivialName_List = row[1].split(',')
            TrivialName_List = re.split(r'[,\/;；]', row[1])
        corpus_Ingre.append(row[0])

        for tn in TrivialName_List:
            if tn != "":
                corpus_Ingre.append(tn)

    corpus_Ingre = set(corpus_Ingre)
    corpus_Ingre = list(corpus_Ingre)

    corpus_All = corpus_All + corpus_Ingre
    conn.commit()
    return 
getData()
# 文字檢索演算法
def BM25(keyword,corpus_data):
    tokenized_corpus = [list(jieba.cut(doc)) for doc in corpus_data]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = list(jieba.cut(keyword))
    doc_scores = bm25.get_scores(tokenized_query)
    top10 = bm25.get_top_n(tokenized_query,corpus_data,n=10)

    # print("斷詞結果:",tokenized_corpus)

    list_doc_scores = list(doc_scores)
    list_corpus = list(corpus_data)

    #df sort
    dict_df2={"Similarity":list_doc_scores,
             "Name":list_corpus
            }
    data2=DataFrame(dict_df2)
    sult2=data2.sort_values(by=['Similarity'],ascending=False)
    sult2 = sult2.reset_index(drop=True) #reset df index  
    print(sult2.head(10))
    return(sult2)

def TFIDF(keyword,corpus_data):
    class TF_IDF_Model(object):
        def __init__(self, corpus_data):
            self.corpus_data = corpus_data
            self.documents_number = len(corpus_data)
            self.tf = []
            self.idf = {}
            self.init()

        def init(self):
            df = {}
            for document in self.corpus_data:
                temp = {}
                for word in document:
                    temp[word] = temp.get(word, 0) + 1/len(document)
                self.tf.append(temp)
                for key in temp.keys():
                    df[key] = df.get(key, 0) + 1
            for key, value in df.items():
                self.idf[key] = np.log(self.documents_number / (value + 1))

        def get_score(self, index, keyword):
            score = 0.0
            for q in keyword:
                if q not in self.tf[index]:
                    continue
                score += self.tf[index][q] * self.idf[q]
            return score

        def get_documents_score(self, keyword):
            score_list = []
            for i in range(self.documents_number):
                score_list.append(self.get_score(i, keyword))
            return score_list
        
    tf_idf_model = TF_IDF_Model(corpus_data)
    scores = tf_idf_model.get_documents_score(keyword)

    #df sort
    dict_df4={"Similarity":scores,
             "Name":corpus_data
            }
    data4=DataFrame(dict_df4)
    sult4=data4.sort_values(by=['Similarity'],ascending=False)
    sult4 = sult4.reset_index(drop=True) #reset df index
    print(sult4.head(10))
    return(sult4)

def Synonyms(keyword,corpus_data):
    Similarity_list=[]
    name_list=[]

    i=-1
    for abc in corpus_data:
        i=i+1
        r = synonyms.compare(corpus_data[i], keyword, seg=False)
        Similarity_list.append(r)
        name_list.append(corpus_data[i])

    dict_df5={"Similarity":Similarity_list,
             "Name":name_list
            }
    data5=DataFrame(dict_df5)
    sult5=data5.sort_values(by=['Similarity'],ascending=False)
    sult5 = sult5.reset_index(drop=True) #reset df index
    print(sult5.head(10))
    return(sult5)

def Algorithm3_Vote_Top1(keyword,corpus_data):
    np.seterr(divide='ignore',invalid='ignore')
    df_top1 = pd.concat([BM25(keyword,corpus_data)[0:3],TFIDF(keyword,corpus_data)[0:3],Synonyms(keyword,corpus_data)[0:3]],axis=0)
    freq_top1 = df_top1.groupby(['Name']).count() 
    freq_top1 = freq_top1.sort_values(by=['Similarity'],ascending=False) #sort
    freq_top1 = freq_top1.reset_index()
    return(freq_top1)
def BM25_Ingre(Name):
    Result = 0
    corpus_get = []
    flag = 0
    cursor.execute('SELECT Id FROM `Ingre` where Name = \'' + Name + '\' or TrivialName like  \'' + Name + '\'')
    cursor_getIngre = cursor.fetchone()
    if cursor_getIngre is not None:
        Result = cursor_getIngre[0]
    else:
        cursor.execute('SELECT Id, Name, TrivialName FROM `Ingre` where Name like \'' + Name + '\' or TrivialName like  \'%' + Name + '%\'')
        flag_getName = 0
        for row in cursor:
            Name_List = []
            Name_List.append(row[1])
            Name_List += re.split(r'[,\/;]', row[2])
            for Name_count in Name_List:
                if Name_count == Name:
                    flag_getName += 1
                    if flag_getName == 1:
                        Result = row[0]      
        if Result == 0:
            df = Algorithm5_Vote_Top1(Name,corpus_Ingre)
            getName = df.iloc[0].at['Name']
            cursor.execute('SELECT Id, Name, TrivialName FROM `Ingre` where Name like \'' + getName + '\' or TrivialName like  \'%' + getName + '%\'')
            flag_getName = 0
            for row in cursor:
                Name_List = []
                Name_List.append(row[1])
                try:
                    Name_List += re.split(r'[,\/;]', row[2])
                except:
                    pass
                for Name_count in Name_List:
                    if Name_count == getName:
                        flag_getName += 1
                        if flag_getName == 1:
                            Result = row[0]      
    return Result

In [25]:
class getIdObject:
    def __init__(self, Id, Name, Content):
        self.Id = Id
        self.Name = Name
        self.Content = Content
def getNameAndId(Name):
    Id = ""

    cursor.execute('SELECT Id FROM `Recipe` where Name = \'' + Name + '\'')
    get_Id_value = cursor.fetchone()
    Content = False
    if get_Id_value is None:
        cursor.execute('SELECT Id, Name, TrivialName FROM `Ingre` where Name = N\'' + Name  + '\' or TrivialName like  \'%' + Name + '%\'')
        flag_getName = 0
        for row in cursor:
            Name_List = []
            Name_List.append(row[1])
            try:
                Name_List += re.split(r'[,\/;]', row[2])
            except:
                pass
            for Name_count in Name_List:
                if Name_count == Name:
                    flag_getName += 1
                    if flag_getName == 1:
                        get_Id_value = row[0] 
                        Content = True
    if Content:
        getId_Json = getIdObject(get_Id_value,Name,Content)
    else:
#         print(get_Id_value)
        getId_Json = getIdObject(get_Id_value[0],Name,Content)
    Json = json.dumps(getId_Json.__dict__,ensure_ascii=False)
    return Json
def get10Match(Out_lists_Json):
    # Out_lists_Json = json.loads(Out_lists_Json)   
    lists_mult = ""
    lists_IdAndName = []
    
    for i in Out_lists_Json:
        for item_name in i['Name']:
            if item_name['Pos'] != "Neu" and item_name['Pos'] != "Nf" and item_name['Pos'] != "Neqa" and item_name['Pos'] != "Nd" and item_name['Pos'] != "Di" and item_name['Pos'] != "Nh" and item_name['label'] != "吃" and item_name['label'] != "喝" and item_name['label'] != "早餐" and item_name['label'] != "午餐" and item_name['label'] != "晚餐" and item_name['label'] != "了":
                lists_mult += item_name['label']
 
    df = Algorithm3_Vote_Top1(lists_mult,corpus_All)

    for count in range(len(df)):
        Name = df.iloc[count].at['Name']
        lists_IdAndName.append(json.loads(getNameAndId(Name)))

    # Out_lists_Json = json.dumps(Out_lists_Json, ensure_ascii=False)
    return lists_IdAndName
get10Match(WSandPOS_result)

   Similarity       Name
0   15.455282       炒空心菜
1   13.385497    豆腐乳炒空心菜
2   11.127140        空心菜
3   10.557704  干貝XO醬炒空心菜
4    9.406562      清炒空心菜
5    9.406562      美國空心菜
6    6.048720        炒莧菜
7    6.048720         炒蝦
8    6.048720       炒綠豆芽
9    6.048720        炒蛤蜊
   Similarity     Name
0    4.723928      空心菜
1    4.582495     炒空心菜
2    3.665996    清炒空心菜
3    3.603739       菜心
4    2.834357    美國空心菜
5    2.618569  豆腐乳炒空心菜
6    2.402493      包心菜
7    2.402493      油菜心
8    2.402493      鵝菜心
9    2.402493      芥菜心
   Similarity     Name
0       1.000     炒空心菜
1       0.928    清炒空心菜
2       0.926      空心菜
3       0.900    美國空心菜
4       0.883  豆腐乳炒空心菜
5       0.866      無心菜
6       0.864      捲心菜
7       0.855      炒莧菜
8       0.854      包心菜
9       0.854      炒青菜


[{'Id': 2427, 'Name': '炒空心菜', 'Content': False},
 {'Id': 1180, 'Name': '空心菜', 'Content': True},
 {'Id': 2891, 'Name': '清炒空心菜', 'Content': False},
 {'Id': 2346, 'Name': '豆腐乳炒空心菜', 'Content': False}]

In [30]:
Result_Food = {'Id': 2427, 'Name': '炒空心菜', 'Content': False}

In [31]:
class BM25ContentResultFormat:
    def __init__(self, Name, Weight, Ingre_Id):
        self.Name = Name
        self.Weight = Weight      
        self.Ingre_Id = Ingre_Id 
def getContents(getContent_lists_json):
    item = getContent_lists_json
    Id = item['Id']
    Name = item['Name']
    Content_Lists = []
    WeightTotal = 0
    if not item['Content']:
        cursor.execute('SELECT * FROM Recipe_Item where Recipe_Id = \'' + str(Id) + '\'')
        get_all_cursor = cursor.fetchall()
        for row in get_all_cursor:
            if row[4] is not None:
                BM25Id = row[4]
                WeightTotal += row[3]
            else:
                BM25Id = BM25_Ingre(row[2])
                print(BM25Id)
                WeightTotal += row[3]
            BM25ContentResult = BM25ContentResultFormat(row[2], row[3], BM25Id)
            
            BM25ContentResult_Json = json.dumps(BM25ContentResult.__dict__,ensure_ascii=False)
            BM25ContentResult_Json = json.loads(BM25ContentResult_Json)
            
            Content_Lists.append(BM25ContentResult_Json)
    item['Weight'] = WeightTotal
    item['Food_Contents'] = Content_Lists
    # getContent_lists_json = json.dumps(getContent_lists_json, ensure_ascii=False)
    return getContent_lists_json
getContents_result = getContents(Result_Food)
getContents(Result_Food)

3730
3730


{'Id': 2427,
 'Name': '炒空心菜',
 'Content': False,
 'Weight': 307.0,
 'Food_Contents': [{'Name': '米酒', 'Weight': 15.0, 'Ingre_Id': 3730},
  {'Name': '空心菜', 'Weight': 250.0, 'Ingre_Id': 1180},
  {'Name': '高纖味精', 'Weight': 5.0, 'Ingre_Id': 673},
  {'Name': '蒜', 'Weight': 32.0, 'Ingre_Id': 1753},
  {'Name': '鹽', 'Weight': 5.0, 'Ingre_Id': 671}]}

In [28]:
def CALC_cal(getContents_result):
    Total_cal  = 0
    for w in getContents_result['Food_Contents']:
        Ingre_cal = 0
        cursor.execute('SELECT Id, Name, Protein FROM Ingre where Id = \'' + str(w['Ingre_Id']) + '\'')
        get_Ingre = cursor.fetchone()
        Ingre_cal = round(w['Weight'] * (get_Ingre[2]/100),2) # 食材個別熱量(資料庫中以每100克為單位，因此需要/100)，並四捨五入
        Total_cal += Ingre_cal
        print(Ingre_cal)
    print("若依照食譜中的份量計算之熱量："+str(Total_cal))
    Total_cal = round(Total_cal/getContents_result['Weight']*Total_Weight,2)
    print("若依照使用者攝入的份量計算之熱量："+str(Total_cal))
    return
CALC_cal(getContents_result)

0.0
0.23
3.27
1.07
0.31
23.65
0.0
若依照食譜中的份量計算之熱量：28.529999999999998
若依照使用者攝入的份量計算之熱量：40.76


In [29]:
# 模擬數據，繪製可視化圖表
import pandas as pd
import plotly.express as px

Record = [
    ["20220606", "鹹豬肉炒高麗菜", 230.8, 9.18, 11.09, 17.21],
    ["20220606", '百香戚風', 366.7625540277505, 15.993482526297573, 53.84245350743413, 10.21872191782975],
    ["20220606", '牛肉炒飯', 299.5872491187034, 15.475610698249818, 52.80104284335613, 2.990803074029922],
    ["20220606", '香港風格午餐肉麵', 305.5147564931946, 5.086214853414154, 42.92756974464416, 11.473865000727844],
    ["20220606", '草莓奶茶', 127.4202, 4.959, 22.9959, 1.7334],
    ["20220606", '咖哩馬鈴薯', 97.06205383978174, 3.510099600604057, 13.595384721530918, 1.367012523200989],
    ["20220606", '菜脯蛋', 328.34989000669765, 22.72861292126465, 3.800571442083359, 13.778440978027342],
    
]

Record_df = pd.DataFrame(Record)
Record_df.columns = ["Date", "名稱", "熱量", "蛋白質", "碳水化合物", "脂肪"] 
Record_df

,Date,名稱,熱量,蛋白質,碳水化合物,脂肪
0,20220606,鹹豬肉炒高麗菜,230.800000,9.180000,11.090000,17.210000
1,20220606,百香戚風,366.762554,15.993483,53.842454,10.218722
2,20220606,牛肉炒飯,299.587249,15.475611,52.801043,2.990803
3,20220606,香港風格午餐肉麵,305.514756,5.086215,42.927570,11.473865
4,20220606,草莓奶茶,127.420200,4.959000,22.995900,1.733400
5,20220606,咖哩馬鈴薯,97.062054,3.510100,13.595385,1.367013
6,20220606,菜脯蛋,328.349890,22.728613,3.800571,13.778441


In [124]:
fig = px.pie(Record_df, values='熱量', names='名稱', title='當天熱量來源', hole=.3)
fig.show()

In [155]:
Record_Week = [
    ["20220606", 1540.8, 9.18, 11.09, 17.21, 1600],
    ["20220607", 1700.7625540277505, 15.993482526297573, 53.84245350743413, 10.21872191782975, 1600],
    ["20220608", 1405.5872491187034, 15.475610698249818, 52.80104284335613, 2.990803074029922, 1600],
    ["20220609", 1785.5147564931946, 5.086214853414154, 42.92756974464416, 11.473865000727844, 1600],
    ["20220610", 1600.4202, 4.959, 22.9959, 1.7334, 1600],
    ["20220611", 1785.06205383978174, 3.510099600604057, 13.595384721530918, 1.367012523200989, 1600],
    ["20220612", 1560.34989000669765, 22.72861292126465, 3.800571442083359, 13.778440978027342, 1600],
    ["20220613", 1689.8, 9.18, 11.09, 17.21, 1600],
    ["20220614", 1700.7625540277505, 15.993482526297573, 53.84245350743413, 10.21872191782975, 1600],
    ["20220615", 1405.5872491187034, 15.475610698249818, 52.80104284335613, 2.990803074029922, 1600],
    ["20220616", 1785.5147564931946, 5.086214853414154, 42.92756974464416, 11.473865000727844, 1600],
    ["20220617", 1600.4202, 4.959, 22.9959, 1.7334, 1600],
    ["20220618", 1785.06205383978174, 3.510099600604057, 13.595384721530918, 1.367012523200989, 1600],
    ["20220619", 1560.34989000669765, 22.72861292126465, 3.800571442083359, 13.778440978027342, 1600],
    ["20220620", 1584.8, 9.18, 11.09, 17.21, 1600],
    ["20220621", 1700.7625540277505, 15.993482526297573, 53.84245350743413, 10.21872191782975, 1600],
    ["20220622", 1405.5872491187034, 15.475610698249818, 52.80104284335613, 2.990803074029922, 1600],
    ["20220623", 1608.5147564931946, 5.086214853414154, 42.92756974464416, 11.473865000727844, 1600],
    ["20220624", 1600.4202, 4.959, 22.9959, 1.7334, 1600],
    ["20220625", 1785.06205383978174, 3.510099600604057, 13.595384721530918, 1.367012523200989, 1600],
    ["20220626", 1560.34989000669765, 22.72861292126465, 3.800571442083359, 13.778440978027342, 1600],
    
]

Record_Week_df = pd.DataFrame(Record_Week)
Record_Week_df.columns = ["Date", "熱量", "蛋白質", "碳水化合物", "脂肪", 'Standard'] 
Record_Week_df

,Date,熱量,蛋白質,碳水化合物,脂肪,Standard
0,20220606,1540.800000,9.180000,11.090000,17.210000,1600
1,20220607,1700.762554,15.993483,53.842454,10.218722,1600
2,20220608,1405.587249,15.475611,52.801043,2.990803,1600
3,20220609,1785.514756,5.086215,42.927570,11.473865,1600
4,20220610,1600.420200,4.959000,22.995900,1.733400,1600
5,20220611,1785.062054,3.510100,13.595385,1.367013,1600
6,20220612,1560.349890,22.728613,3.800571,13.778441,1600
7,20220613,1689.800000,9.180000,11.090000,17.210000,1600
8,20220614,1700.762554,15.993483,53.842454,10.218722,1600
9,20220615,1405.587249,15.475611,52.801043,2.990803,1600


In [183]:
plot = px.line(Record_Week_df, x = 'Date', 
               y = ['熱量', 'Standard'], ) 
# plot = px.line(Record_Week_df, x = 'Date', 
#                y = 'Standard', ) 
plot.show()